In [1]:
#### CRITICAL - ENABLE GPU

import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

## Mount Drive into Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 11.7 MB/s eta 0:00:00


In [3]:
## PyTorch Transformer
from pytorch_transformers import RobertaModel, RobertaTokenizer
from pytorch_transformers import RobertaForSequenceClassification, RobertaConfig

In [4]:
## Check if Cuda is Available
print(torch.cuda.is_available())

True


In [5]:
## Install PyTorch-Transformer

In [6]:
!pip install -U pytorch-transformers

In [7]:
## Importing Datasets

In [8]:
'''
Important Step - Make sure you upload the data file to the exact location below. If you uploaded correctlt, the following command will run
'''

!ls drive/'My Drive'/2017-06-custom-intent-engines

AddToPlaylist	GetWeather  RateBook   SearchCreativeWork
BookRestaurant	PlayMusic   README.md  SearchScreeningEvent


In [9]:
'''
Create the Dataset Path
'''


dataset_path = "drive/My Drive/2017-06-custom-intent-engines/"

In [10]:

'''
***Explain*** Summarize, in bullet points, what is the code doing?.
'''

dataset = pd.DataFrame(columns = ['utterance', 'label'])
for intent in ['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork',
               'SearchScreeningEvent']:
    with open(dataset_path + intent + "/train_" + intent + ".json",
              encoding='cp1251') as data_file:
        data = json.load(data_file)
    print("Class: {}, # utterances: {}".format(intent,len(data[intent])))
    texts = []
    for i in range(len(data[intent])):
        text = ''
        for j in range(len(data[intent][i]['data'])):
            text += data[intent][i]['data'][j]['text']
        dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
dataset.tail()

Class: AddToPlaylist, # utterances: 300


<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_

Class: BookRestaurant, # utterances: 300


<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_

Class: GetWeather, # utterances: 300


...

Class: SearchCreativeWork, # utterances: 300


<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_

Class: SearchScreeningEvent, # utterances: 300


<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
<ipython-input-10-9812e55e45e2>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'utterance': text, 'label': intent}, ignore_

,utterance,label
2095,Is Across the Line playing at the closest movi...,SearchScreeningEvent
2096,Which animated movies are playing in the neigh...,SearchScreeningEvent
2097,Where is They Always Return at Dawn playing,SearchScreeningEvent
2098,What is the movie schedule in the neighborhood,SearchScreeningEvent
2099,Tell me when Howling II: Your Sister Is a Were...,SearchScreeningEvent


In [11]:
'''
Assigning an Index to each intent. We will use this later
'''

'''
***Explain*** Why do we convert labels to indexes?.
'''

label_to_ix = {}
for label in dataset.label:
    for word in label.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)
label_to_ix

{'AddToPlaylist': 0,
 'BookRestaurant': 1,
 'GetWeather': 2,
 'PlayMusic': 3,
 'RateBook': 4,
 'SearchCreativeWork': 5,
 'SearchScreeningEvent': 6}

In [12]:
## Loading RoBERTa classes

In [13]:
config = RobertaConfig.from_pretrained('roberta-base')
config.num_labels = len(list(label_to_ix.values()))
config

100%|██████████| 481/481 [00:00<00:00, 465818.57B/s]


{
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 7,
  "output_attentions": false,
  "output_hidden_states": false,
  "pad_token_id": 1,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

In [14]:
'''
Loading Pretrained tokenizer and instantiating the model from settings in config
'''

'''
***Explain*** : a. What is a tokenizer? b. What is special about the following tokenizer?.
'''

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

'''
***Explain*** :  What is the next line doing?
'''

model = RobertaForSequenceClassification(config)

100%|██████████| 456318/456318 [00:00<00:00, 6061249.19B/s]


In [15]:
## Feature Preparation

In [17]:
'''
Some important Feature Engineering
'''

'''
***Explain*** : What are the implications for setting  include_CLS_token = True, include_SEP_token = True ?
'''

def prepare_features(seq_1, max_seq_length = 300,
             zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [18]:
msg = "My dog is cute!"
prepare_features(msg)

(tensor([[    0,  1308,  2335,    16, 11962,   328,     2]]),
 [1, 1, 1, 1, 1, 1, 1])

In [19]:
## Dataset Loader Classes

In [20]:
class Intents(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe

    def __getitem__(self, index):
        utterance = self.data.utterance[index]
        label = self.data.label[index]
        X, _  = prepare_features(utterance)
        y = label_to_ix[self.data.label[index]]
        return X, y

    def __len__(self):
        return self.len

In [21]:
train_size = 0.8
train_dataset=dataset.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_dataset=dataset.drop(train_dataset.index).reset_index(drop=True)

In [22]:
print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (2100, 2)
TRAIN Dataset: (1680, 2)
TEST Dataset: (420, 2)


In [23]:
training_set = Intents(train_dataset)
testing_set = Intents(test_dataset)

In [24]:
training_set.__getitem__(0)[0].shape

torch.Size([1, 8])

In [25]:
model(training_set.__getitem__(0)[0])

(tensor([[ 0.5795, -0.0908, -0.1052,  0.3896, -0.0656, -0.4291, -0.0250]],
        grad_fn=<AddmmBackward0>),)

In [26]:
## Training Params

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.cuda()

In [28]:
# Parameters
params = {'batch_size': 1,
          'shuffle': True,
          'drop_last': False,
          'num_workers': 1}

In [29]:
training_loader = DataLoader(training_set, **params)
testing_loader = DataLoader(testing_set, **params)

In [30]:
'''
Instantiate the Loss
'''
'''
***Explain*** why cross entropy loss?, also print the model and explain why are not we using softmax at the end?
'''
loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05
optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,)

In [31]:
## Test Forward Pass
inp = training_set.__getitem__(0)[0].cuda()
output = model(inp)[0]
print(output.shape)

torch.Size([1, 7])


In [32]:
torch.__version__

'2.1.0+cu118'

In [33]:
'''
Actually train the model with train data
'''
'''
***Explain*** the Training Code Chunk in detail. Especially what is torch.max() doing here?
'''



max_epochs = 3
model = model.train()
for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        optimizer.zero_grad()
        sent = sent.squeeze(0)
        if torch.cuda.is_available():
          sent = sent.cuda()
          label = label.cuda()
        output = model.forward(sent)[0]
        _, predicted = torch.max(output, 1)

        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()

        if i%100 == 0:
            correct = 0
            total = 0
            for sent, label in testing_loader:
                sent = sent.squeeze(0)
                if torch.cuda.is_available():
                  sent = sent.cuda()
                  label = label.cuda()
                output = model.forward(sent)[0]
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted.cpu() == label.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

<ipython-input-33-54af028524d3>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(max_epochs)):


  0%|          | 0/3 [00:00<?, ?it/s]

EPOCH -- 0
Iteration: 0. Loss: 1.8194282054901123. Accuracy: 0.0%
Iteration: 100. Loss: 1.479205846786499. Accuracy: 2.380952380952381%
Iteration: 200. Loss: 2.16001033782959. Accuracy: 0.23809523809523808%
Iteration: 300. Loss: 1.5551180839538574. Accuracy: 28.095238095238095%
Iteration: 400. Loss: 1.8825359344482422. Accuracy: 53.333333333333336%
Iteration: 500. Loss: 2.8045523166656494. Accuracy: 28.571428571428573%
Iteration: 600. Loss: 1.616103172302246. Accuracy: 64.28571428571429%
Iteration: 700. Loss: 0.8027922511100769. Accuracy: 44.523809523809526%
Iteration: 800. Loss: 0.6654515266418457. Accuracy: 75.23809523809524%
Iteration: 900. Loss: 0.07756974548101425. Accuracy: 56.666666666666664%
Iteration: 1000. Loss: 0.10547803342342377. Accuracy: 47.857142857142854%
Iteration: 1100. Loss: 0.05533233657479286. Accuracy: 88.80952380952381%
Iteration: 1200. Loss: 0.12229453027248383. Accuracy: 93.33333333333333%
Iteration: 1300. Loss: 0.20386968553066254. Accuracy: 91.42857142857143

In [34]:
'''
***Explain*** what is the get_reply function doing?
'''
def get_reply(msg):
  model.eval()
  input_msg, _ = prepare_features(msg)
  if torch.cuda.is_available():
    input_msg = input_msg.cuda()
  output = model(input_msg)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction=list(label_to_ix.keys())[pred_label]
  return prediction

In [35]:
label_to_ix.keys()

dict_keys(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork', 'SearchScreeningEvent'])

In [36]:
'''Different text sentences pass to the model'''

get_reply("play radiohead song")

'PlayMusic'

In [37]:
get_reply("it is rainy in Sao Paulo")

'GetWeather'

In [38]:
get_reply("sun shines all day")

'PlayMusic'

In [39]:
get_reply("low humidity, high altitude")

'GetWeather'

In [40]:
get_reply("Book tacos for me tonight")

'BookRestaurant'

In [41]:
get_reply("Book a table for me tonight")

'BookRestaurant'

In [42]:
get_reply("I want BBQ tonight")

'PlayMusic'

## Custom sentences

In [43]:
get_reply("Add the song Radioactive to my favorites playlist")

'AddToPlaylist'

In [48]:
get_reply("Find me a sushi restaurant")

'SearchCreativeWork'

In [45]:
get_reply("I really enjoyed The Lord of the Rings")

'SearchCreativeWork'

In [46]:
get_reply("My favorite book is Mistborn")

'GetWeather'

In [47]:
get_reply("What is the weather near me?")

'GetWeather'